### 4.2 user_answer 特征处理


In [ ]:
import pandas as pd
import os
import numpy as np

data_path = '../../../data'
answer_info = pd.read_csv(os.path.join(data_path, 'answer_info_0926.txt'), header=None, sep='\t')
#train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')

#train1.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
answer_info.columns = ['回答id', '问题id', '用户id', '回答创建时间', '回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数', '点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数']

drop_feat = ['回答内容的单字编码序列','回答内容的切词编码序列']
answer_info  = answer_info.drop(drop_feat, axis=1)

#user_answer_info = pd.merge(train1, answer_info, how='left', on='用户id')

In [ ]:
answer_info['回答创建时间-hour'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
answer_info['回答创建时间-day'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
answer_info['回答创建时间-wkday'] = answer_info['回答创建时间-day']%7

In [ ]:
print(answer_info['回答创建时间-day'].min())

In [ ]:
# 减少内存占用
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')
train1.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
train1['邀请创建时间-day'] = train1['邀请创建时间'].apply(lambda x:x.split('-')[0].split('D')[1])
train1['邀请创建时间-hour'] = train1['邀请创建时间'].apply(lambda x:x.split('-')[1].split('H')[1])

answer_info = pd.merge(answer_info, train1, how='left', on=['用户id','问题id'])

In [ ]:
print(answer_info.info())

In [ ]:
answer_info = answer_info.fillna({'是否回答':1, '邀请创建时间-day':-100,'邀请创建时间-hour':-100})

In [ ]:
print(1)

In [ ]:
answer_info['邀请创建时间-hour'] = answer_info['邀请创建时间-hour'].astype(int)
answer_info['邀请创建时间-day'] = answer_info['邀请创建时间-day'].astype(int)

In [ ]:
# t3 = answer_info
# t3 = answer_info.groupby('用户id')['邀请创建时间-hour'].agg('value_counts').to_frame()
# t3.columns = ['u_i_hr_count']
# t3 = t3.reset_index()
# answer_info = pd.merge(answer_info, t3, on=['用户id','邀请创建时间-hour'], how='left')

In [ ]:
def extract_user_features(answer_info, train_end, etype):
    
    if etype=='val':
        days = 21
    else:
        days = 14
    
    answer_info['u_a_last_daytime'] = answer_info['用户id'].map(answer_info['回答创建时间-day'].groupby(answer_info['用户id']).max()) #为每个特征，匹配上其每个值对应到的 该值出现的次数

########## 在baseline里已写  ##########
#     feat_cols = ['点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数','回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数']
#     for feat in feat_cols:
#         answer_info['用户总'+feat+'_last3week'] = answer_info['用户id'].map(answer_info[feat].groupby(answer_info['用户id']).sum())
#         answer_info['用户平均'+feat+'_last3week'] = answer_info['用户id'].map(answer_info[feat].groupby(answer_info['用户id']).mean()).astype(int)
#         answer_info['用户最大'+feat+'_last3week'] = answer_info['用户id'].map(answer_info[feat].groupby(answer_info['用户id']).max())
#     print("feat_cols stastics features loaded.")
#     print(answer_info.info())
    
#     for feat in feat_cols:
#         answer_info['用户总'+feat+'_last3day'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-3][feat].groupby(answer_info['用户id']).sum())
#         answer_info['用户平均'+feat+'_last3day'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-3][feat].groupby(answer_info['用户id']).mean())
#         answer_info['用户最大'+feat+'_last3day'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-3][feat].groupby(answer_info['用户id']).max())
#     print("feat_cols stastics features loaded.")
#     print(answer_info.info())

#     for feat in feat_cols:
#         answer_info['用户总'+feat+'_last2week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-14][feat].groupby(answer_info['用户id']).sum())
#         answer_info['用户平均'+feat+'_last2week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-14][feat].groupby(answer_info['用户id']).mean())
#         answer_info['用户最大'+feat+'_last2week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-14][feat].groupby(answer_info['用户id']).max())
#     print("feat_cols stastics features loaded.")
#     print(answer_info.info())    
    
#     for feat in feat_cols:
#         answer_info['用户总'+feat+'_last1week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-7][feat].groupby(answer_info['用户id']).sum())
#         answer_info['用户平均'+feat+'_last1week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-7][feat].groupby(answer_info['用户id']).mean())
#         answer_info['用户最大'+feat+'_last1week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-7][feat].groupby(answer_info['用户id']).max())
#     print("feat_cols stastics features loaded.")
#     print(answer_info.info())    
        
    answer_info['u_a_num'] = answer_info['用户id'].map(answer_info['回答id'].groupby(answer_info['用户id']).nunique())
    print("用户关联回答数 laoded")
#    print(answer_info.info())
    
#     t1 = answer_info.groupby('用户id')['是否回答'].agg(['mean', 'sum', 'std', 'count']).reset_index()  
#     t1.columns = ['用户id', 'u_i_mean', 'u_i_sum', 'u_i_std', 'u_i_count']
#     t1 = t1.fillna({'用户邀请方差':-1})
#     t1.loc[:, '用户邀请方差'] += 1
#     answer_info = pd.merge(answer_info, t1, on='用户id', how='left')
#     del t1
#     print("user label features loaded.")
#     print(answer_info.info())
    
    answer_info['u_ai_diff_hour'] = answer_info['回答创建时间-hour'] - answer_info['邀请创建时间-hour']
    answer_info.loc[answer_info['u_ai_diff_hour']>=100, 'u_ai_diff_hour'] = 30
    answer_info.loc[:, 'u_ai_diff_hour'] += 24
    t2 = answer_info.loc[answer_info['u_ai_diff_hour']<54, :]
    t2 = t2.groupby('用户id')['u_ai_diff_hour'].agg(['mean', 'sum', 'max', 'min']).reset_index()  
    t2.columns = ['用户id', 'u_a_i_diffhour_mean', 'u_a_i_diffhour_sum', 'u_a_i_diffhour_max', 'u_a_i_diffhour_min']
    answer_info = pd.merge(answer_info, t2, on='用户id', how='left')
    del t2
    print("u_a_i_diff_hour feature loaded.")
#    print(answer_info.info())
    
    answer_info['u_ai_diff_day'] = answer_info['回答创建时间-day'] - answer_info['邀请创建时间-day']
    answer_info.loc[answer_info['u_ai_diff_day']>=3900, 'u_ai_diff_day'] = 100
    t3 = answer_info.loc[answer_info['u_ai_diff_day']<100, :]
    t3 = t3.groupby('用户id')['u_ai_diff_day'].agg(['mean', 'sum', 'max', 'min']).reset_index()  
    t3.columns = ['用户id', 'u_a_i_diffday_mean', 'u_a_i_diffday_sum', 'u_a_i_diffday_max', 'u_a_i_diffday_min']
    answer_info = pd.merge(answer_info, t3, on='用户id', how='left')
    del t3
    print("u_a_i_diff_day feature loaded.")
#    print(answer_info.info())
    
    answer_info['u_a_mean_hour'] = answer_info['用户id'].map(answer_info['回答创建时间-hour'].groupby(answer_info['用户id']).agg('mean'))
    answer_info['u_a_mean_hour_last3day'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-3]['回答创建时间-hour'].groupby(answer_info['用户id']).agg('mean'))
    answer_info['u_a_mean_hour_last1week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-7]['回答创建时间-hour'].groupby(answer_info['用户id']).agg('mean'))

    answer_info['u_a_mean_hour_last2week'] = answer_info['用户id'].map(answer_info[answer_info['邀请创建时间-day']>train_end-days]['回答创建时间-hour'].groupby(answer_info['用户id']).agg('mean')) 
    
    t4 = answer_info.groupby('用户id')['回答创建时间-hour'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()
    t4.columns = ['用户id', 'u_a_most_hour']
    t4['u_a_most_hour'] = t4['u_a_most_hour'].astype(int)
    answer_info = pd.merge(answer_info, t4, on='用户id', how='left')
    del t4
    print("用户习惯回答时间-hour loaded.")
#    print(answer_info.info())
    
    t5 = answer_info[answer_info['邀请创建时间-day']>train_end-3].groupby('用户id')['回答创建时间-hour'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()
    t5.columns = ['用户id', 'u_a_most_hour_last3day']
    t5['u_a_most_hour_last3day'] = t5['u_a_most_hour_last3day'].astype(int)
    answer_info = pd.merge(answer_info, t5, on='用户id', how='left')
    del t5
    print("用户习惯回答时间-hour_last3day loaded.")
#    print(answer_info.info())    
    
    t5 = answer_info[answer_info['邀请创建时间-day']>train_end-7].groupby('用户id')['回答创建时间-hour'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()
    t5.columns = ['用户id', 'u_a_most_hour_last1week']
    t5['u_a_most_hour_last1week'] = t5['u_a_most_hour_last1week'].astype(int)
    answer_info = pd.merge(answer_info, t5, on='用户id', how='left')
    del t5
    print("用户习惯回答时间-hour_last1week loaded.")
#    print(answer_info.info())    
    
    t5 = answer_info[answer_info['邀请创建时间-day']>train_end-days].groupby('用户id')['回答创建时间-hour'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()
    t5.columns = ['用户id', 'u_a_most_hour_last2week']
    t5['u_a_most_hour_last2week'] = t5['u_a_most_hour_last2week'].astype(int)
    answer_info = pd.merge(answer_info, t5, on='用户id', how='left')
    del t5
    print("用户习惯回答时间-hour_last2week loaded.")
#    print(answer_info.info())    
    
    t5 = answer_info.groupby('用户id')['回答创建时间-wkday'].agg(lambda x: np.mean(pd.Series.mode(x))).reset_index()
    t5.columns = ['用户id', 'u_a_most_wkday']
    t5['u_a_most_wkday'] = t5['u_a_most_wkday'].astype(int)
    answer_info = pd.merge(answer_info, t5, on='用户id', how='left')
    del t5
    print("用户习惯回答时间-wkday loaded.")
#    print(answer_info.info())    
   
    t1 = answer_info[answer_info['邀请创建时间-day']>train_end-3]
    t1['u_a_num_last3'] = t1['用户id'].map(t1['回答id'].groupby(t1['用户id']).nunique())
    t1 = t1[['用户id','u_a_num_last3']].drop_duplicates(subset=None, keep='first', inplace=False)
    answer_info = pd.merge(answer_info, t1, on='用户id', how='left')
#    print(answer_info.info())
    del t1
    
    t1 = answer_info[answer_info['邀请创建时间-day']>train_end-7]
    t1['u_a_num_last7'] = t1['用户id'].map(t1['回答id'].groupby(t1['用户id']).nunique())
    t1 = t1[['用户id','u_a_num_last7']].drop_duplicates(subset=None, keep='first', inplace=False)
    answer_info = pd.merge(answer_info, t1, on='用户id', how='left')
    del t1
    
    t1 = answer_info[answer_info['邀请创建时间-day']>train_end-days]
    t1['u_a_num_last14'] = t1['用户id'].map(t1['回答id'].groupby(t1['用户id']).nunique())
    t1 = t1[['用户id','u_a_num_last14']].drop_duplicates(subset=None, keep='first', inplace=False)
    answer_info = pd.merge(answer_info, t1, on='用户id', how='left')
    del t1
    
    print("用户最近回答数loaded.")
#    print(answer_info.info())

    #answer_info['用户回答时间-hour_count'] = answer_info[['用户id','回答创建时间-hour']].map(answer_info['回答创建时间-hour'].groupby(answer_info['用户id']).value_counts())
    #print(answer_info['回答是否被标优'].groupby(answer_info['用户id'].sum()))
    
    drop_feat = ['回答创建时间-wkday', '邀请创建时间-day', '邀请创建时间-hour','u_ai_diff_day','u_ai_diff_hour','邀请创建时间','是否回答', '问题id','回答id','回答创建时间','回答是否被标优','回答是否被推荐','回答是否被收入圆桌','点赞数','评论数','收藏数','感谢数','回答创建时间-hour','回答创建时间-day','举报数', '没有帮助数', '反对数','回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数','取赞数']
    answer_info  = answer_info.drop(drop_feat, axis=1)
    print("feature droped")
    answer_info.drop_duplicates(subset=None, keep='first', inplace=True)

#     drop_feats = ['用户总点赞数_last3day', '用户平均点赞数_last3day', '用户最大点赞数_last3day', '用户总取赞数_last3day', '用户平均取赞数_last3day', '用户最大取赞数_last3day', '用户总评论数_last3day', '用户平均评论数_last3day', '用户最大评论数_last3day', '用户总收藏数_last3day', '用户平均收藏数_last3day', '用户最大收藏数_last3day', '用户总感谢数_last3day', '用户平均感谢数_last3day', '用户最大感谢数_last3day', '用户总举报数_last3day', '用户平均举报数_last3day', '用户最大举报数_last3day', '用户总没有帮助数_last3day', '用户平均没有帮助数_last3day', '用户最大没有帮助数_last3day', '用户总反对数_last3day', '用户平均反对数_last3day', '用户最大反对数_last3day', '用户总回答是否被标优_last3day', '用户平均回答是否被标优_last3day', '用户最大回答是否被标优_last3day', '用户总回答是否被推荐_last3day', '用户平均回答是否被推荐_last3day', '用户最大回答是否被推荐_last3day', '用户总回答是否被收入圆桌_last3day', '用户平均回答是否被收入圆桌_last3day', '用户最大回答是否被收入圆桌_last3day', '用户总是否包含图片_last3day', '用户平均是否包含图片_last3day', '用户最大是否包含图片_last3day', '用户总是否包含视频_last3day', '用户平均是否包含视频_last3day', '用户最大是否包含视频_last3day', '用户总回答字数_last3day', '用户平均回答字数_last3day', '用户最大回答字数_last3day', '用户平均回答时间-hour']
#     train_for_save  = train_for_save.drop(drop_feats, axis=1)

    answer_info = reduce_mem_usage(answer_info)
    print(answer_info.info())

    return answer_info

In [ ]:
train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save = answer_info[answer_info['邀请创建时间-day']>=train_start][answer_info['邀请创建时间-day']<=train_end]
train_for_save = extract_user_features(train_for_save, train_end ,'train')

val_for_save = answer_info[answer_info['邀请创建时间-day']>=val_start][answer_info['邀请创建时间-day']<=val_end]
val_for_save = extract_user_features(val_for_save, val_end, 'val')

In [ ]:
print(answer_info.info())

In [ ]:
print(list(val_for_save.columns.values))

In [ ]:
print(train_for_save.info())

In [ ]:
print(train_for_save['用户id'].nunique())
print(val_for_save['用户id'].nunique())

In [ ]:
print(train_for_save.info())
print(val_for_save.info())

In [ ]:
train_for_save = train_for_save.fillna(-1)
val_for_save = val_for_save.fillna(-1)

In [ ]:
train_for_save.to_hdf('user_answer_train.h5',key='data')
val_for_save.to_hdf('user_answer_val.h5',key='data')

### 分割线-----

In [ ]:
import pandas as pd
import os

info = pd.read_hdf('user_answer.h5', key='data')

In [ ]:
print(info.info())

In [ ]:
print(train_for_save['用户id'].nunique())

In [ ]:
info1 = info[:1000].sort_values(by='用户上次回答时间-day', ascending=True)

In [ ]:
info2 = info1[info1[:1000]['用户上次回答时间-day']<3861]
print(info2)